# Welcome to the FLAME GPU 2 Tutorial!

For the purpose of this tutorial, the Jupyter notebook has been configured to work with FLAME GPU 2. The Jupyter notebook is a web based environment for interactive computing. It is capable of running code in a wide variety of programming languages. The notebook consists of cells which can be of type code or markdown (by default, new cells are created as code cells). You can execute the content of code cells by clicking the Run button. Cells which are actively running code will show a 'Play' icon and cells which have completed will show a green tick.

The notebook will guide you through the various commands required to run and edit a FLAME GPU 2 model using the python interface.

## Before Beginning
If using Google Colab, please ensure you have this notebook enabled with GPU before running (Runtime->Change runtime type). If you are not using Google Colab you can ignore this instruction.

## Running your first code cell
The cell below is a simple example of a code cell. Clicking on the play button which appears when hovering over the cell with the mouse will run the code and you should see "Hello!" displayed beneath the code cell.

In [ ]:
print("Hello!")

# Installing FLAME GPU 2

The python library can be installed through pip. This installs a binary release of the FLAME GPU 2 library. Run the cell below to install the FLAME GPU 2 library.

In [ ]:
import importlib.util
if importlib.util.find_spec('pyflamegpu') is None:
    import sys
    !{sys.executable} -m pip install --extra-index-url https://whl.flamegpu.com/whl/cuda120/ pyflamegpu==2.0.0rc2 # type: ignore

# Import pyflamegpu and some other libraries we will use in the tutorial
import pyflamegpu
import sys, random, math
import matplotlib.pyplot as plt


With the library installed, we need to configure the CUDA_PATH environment variable so the library can find the CUDA installation.

In [ ]:
%env CUDA_PATH=/usr/local/cuda


# The Predator-Prey Model
## Model Description

Predator-prey models capture the dynamic behaviours of competing species within an environment to demonstrate population dynamics as species try to survive. The model is important as it captures emergent macro level phenomenon which can be observed within other domains such as economics.

In its most basic form, a two species system consists of both prey and predators. Prey reproduce but are eaten by predators and predators reproduce and eat prey. Typically such systems can be represented using non linear differential equations known as Lotka-Volterra equations. In a more advanced form of the model, microscopic behaviours of the species can be considered by using agent based representation. E.g. The predators can have energy which is depleted as they move. Prey can form cohesive groups (flocking) and dynamically avoid predators, while predators can move towards prey. If a predator and prey are close enough (i.e. within a fixed kill radius), the predator is able to eat the prey and increase the amount of energy it has. Predators die when they run out of energy.

The model has been implemented in various Agent Based Modelling frameworks (e.g:NetLogo and FLAME). The tutorial model implements the following behaviours:

* Predator and Prey agents move and act in a sequence of iterations (time steps).
* Predator and Prey agents have real valued (x, y) positions and velocities which are integrated over time.
* Predator agents have an initial velocity. A Predator adjusts its velocity to follow the nearest prey within a synthetic limited vision.
* A predator’s energy is reduced by 1 unit of energy each time it moves.
* Prey are eaten by the closest predator agent within a fixed (kill) radius.
* A predator’s energy increases each time it catches a prey.
* A predator dies if it has run out of energy.

More specific details of the model features are described below.

**Prey catching** There is no limit on the amount of prey a predator can eat per iteration. If a predator is close to multiple prey within the specific (kill) radius, the predator will kill/eat them all. A prey cannot be shared between multiple predators. If there are multiple predators within the kill radius then only the closest will eat the prey.

**Reproduction** Each species has a rate of reproduction. When an agent reproduces, the parent’s energy is shared evenly between parent and child, so both will have half of the parent’s energy for the next iteration.


# FLAME GPU 2 Implementation

The FLAME GPU 2 implementation of the predator prey model is based on the assumptions made in the previous section. All agents have a real valued position (``x``, ``y``) within a continuous environment. The environment bounds are wrapped to form a continuous toroidal space in which agents can move. Agents have a velocity (``vx`` and ``vy``) in which they will move and a steering velocity (``steer_x`` and ``steer_y``) which dictates how they should adjust their velocity according to local interactions.Predator agents have a variable (``life``) which contains their level of energy. An agent variable, ``type``, is used for visualisations of the simulation where unique type values correspond to different colours within the visualisation. The type is set by the initial values of the agent and does not change over time.

Each agent has a unique identifier variable (``id``). When a predator kills a prey, it is necessary to communicate identifiers to ensure correct behaviour as will be explained later in this section.

All communication between agents in FLAME GPU 2 is performed indirectly via messages. In this model we have three message types which are required to perform the mechanism for killing. E.g,

* Both prey and predators output their (``x``, ``y``) coordinates to a prey or predator location message respectively.
* Prey read predator locations and perform evasive behaviour to avoid them.
* Predators read prey locations and move towards the closest visible prey.
* Prey read each others location and move towards each other to form cohesive groups.
* Both predators and prey read their own species' locations to avoid each other when in close proximity.
* Prey read the locations of predators and calculate the nearest predator within the kill distance. If a predator is within the kill distance then it notifies this predator by sending a ``prey_eaten_message`` message with the predator's id and the eaten prey dies (is removed for the simulation).
* Predators read the ``prey_eaten_message`` messages and check the to see if their id has been communicated by any dead prey. If the message indicates that the predator ate some prey, then the predator increases its energy/life accordingly.

# The FLAME GPU 2 Model

A FLAME GPU 2 model specification consists of: an **environment** which holds global information related to the simulation such as constant variables and behaviour script file names, **agents** which describe use variables and functions to describe the properties and behaviours of individuals in the simulation, and **messages** which represent information communicated between agents. Each aspect is introduced in the following subsections with examples. If you are confident of the model's behaviour and the FLAMEGPU syntax then feel free to move onto the hands on material.

In the python build of FLAME GPU 2, all of these model elements are described through the python interface. A `ModelDescription` object acts as a container for the various components which make up the model. The following sections will describe how to create each of the above components and attach them to the `ModelDescription`. Each code cell defines a particular function. By defining functions rather than running the code directly, they can be updated individually and defined out of order. These functions are then all called in the correct order in the run simulation cell later in the document.




# Create a Model





The [`ModelDescription`](https://docs.flamegpu.com/guide/2-model-definition/1-model.html#creating-a-modeldescription-object) is created with the name "python-tutorial". Individual functions are used to wrap the creation of each of the various elements of the model.

In [ ]:
def create_model():
    model = pyflamegpu.ModelDescription("python-tutorial")
    return model

# Define Environmental Properties

For the Predator-Prey model a number of self explanatory model parameters are described within the environment section. The first line of the code below queries the model to allow us to modify its environment through the [`EnvironmentDescription`](https://docs.flamegpu.com/guide/2-model-definition/2-environment.html) object, env. Environmental parameters can be added by name and type and are given a default value. These default values may be overwritten by an optional XML or JSON input file describing the initial state of the simulation. This allows modification of model parameters without recompiling the code.

In [ ]:
def define_environment(model):
    """
        Environment
    """
    env = model.Environment()

    # Reproduction
    env.newPropertyFloat("REPRODUCE_PREY_PROB", 0.05)
    env.newPropertyFloat("REPRODUCE_PRED_PROB", 0.03)

    # Cohesion/Avoidance
    env.newPropertyFloat("SAME_SPECIES_AVOIDANCE_RADIUS", 0.035)
    env.newPropertyFloat("PREY_GROUP_COHESION_RADIUS", 0.2)

    # Predator/Prey/Grass interaction
    env.newPropertyFloat("PRED_PREY_INTERACTION_RADIUS", 0.3)
    env.newPropertyFloat("PRED_SPEED_ADVANTAGE", 3.0)
    env.newPropertyFloat("PRED_KILL_DISTANCE", 0.05)
    env.newPropertyFloat("GRASS_EAT_DISTANCE", 0.02)
    env.newPropertyUInt("GAIN_FROM_FOOD_PREY", 80)
    env.newPropertyUInt("GAIN_FROM_FOOD_PREDATOR", 100)
    env.newPropertyUInt("GRASS_REGROW_CYCLES", 100)
    
    # Simulation properties
    env.newPropertyFloat("DELTA_TIME", 0.001)
    env.newPropertyFloat("BOUNDS_WIDTH", 2.0)
    env.newPropertyFloat("MIN_POSITION", -1.0)
    env.newPropertyFloat("MAX_POSITION", 1.0)

# Define Messages

[Messages](https://docs.flamegpu.com/guide/3-behaviour-definition/4-agent-communication.html) are simple collections of variables which are used to communicate data between agents. Messages are defined in the same way as the environment variables and agents, however, messages also require a choice of spatial partitioning scheme. These schemes are typically used to improve computation time by limiting the messages each agent sees to those which are nearby. Spatial partitioning is simple to use in FLAME GPU 2, although we will use the simplest messaging type, MessageBruteForce which features no partitioning. This indicates that the message reading functions will iterate all messages in the list. The code below shows the definition of a brute force message which can be used to communicate the location of a prey agent:

In [ ]:
def define_messages(model):
    """
      Location messages
    """      
    message = model.newMessageBruteForce("predator_location_message")
    message.newVariableID("id")
    message.newVariableFloat("x")
    message.newVariableFloat("y")
        
    message = model.newMessageBruteForce("prey_location_message")
    message.newVariableID("id")
    message.newVariableFloat("x")
    message.newVariableFloat("y")
    

    """
      Agent eaten messages
    """
        
    message = model.newMessageBruteForce("prey_eaten_message")
    message.newVariableID("id")
    message.newVariableInt("pred_id")

    message = model.newMessageBruteForce("grass_eaten_message")
    message.newVariableID("id")
    message.newVariableInt("prey_id")

# Define Agents

**Agent Variables**

[Agents](https://docs.flamegpu.com/guide/2-model-definition/3-agent.html) represent individuals in the simulation and are defined using a `AgentDescription`. Much like the environment, we ask the model to provide us with a reference to the new agent which we can then add variables to. As with the environment, each variable is given a type and a name.

**Specifying Agent Functions**

Agent functions are used to implement agent behaviours. The tables below give the functions of the predators and prey and their roles.

| Prey Function | Description |
|:---|:---|
|prey_output_location | each prey agent outputs information to be read by other agents|
|prey_avoid_pred |	prey agents avoiding predator agents|
|prey_flock | flocking between prey agents|
|prey_move | movement of prey agents|
|prey_eaten | prey agents get killed by predator agents|
|prey_reproduction | regeneration of prey|


| Predator Function | Description |
|:---|:---|
|pred_output_location | each predator agent outputs information to be read by other agents|
|pred_follow_prey | predator agents follow prey agents|
|pred_avoid | predator agents avoid each other|
|pred_move | movement of predator agents|
|pred_eat_or_starve | predator agents gain energy or starve|
|prey_reproduction | regeneration of predators|

Functions can be applied to agents in a specified *initial_state* and can optionally cause agents to move to other *states*. As our model is relatively simple, we only use a single default state. Furthermore, functions can have *conditions*. For example, a function which simulates agent death might check a variable incremented each iteration called life-cycles to see if it has reached a maximum number. Only agents meeting this condition would perform the behaviour and move into a dead state. The predator-prey model is a stateless model (each agent has only a single state) as functions do not require any function conditions. 

Functions can have at most one input and one output. Inputs and outputs are in the form of messages which, are a collection of variables which are persistent until the end of the simulation iteration (at which point they are destroyed). A function description within a model requires that any inputs or outputs are fully specified.

The descriptions of the predator, prey and grass agents' variables and functions in FLAME GPU 2 are given below:

In [ ]:
def define_agents(model):
    """
        Prey agent
    """
    # Create the agent
    agent = model.newAgent("prey")

    # Assign its variables
    agent.newVariableFloat("x")
    agent.newVariableFloat("y")
    agent.newVariableFloat("vx")
    agent.newVariableFloat("vy")
    agent.newVariableFloat("steer_x")
    agent.newVariableFloat("steer_y")
    agent.newVariableInt("life")
    agent.newVariableFloat("type")
    
    # Assign its functions
    fn = agent.newRTCFunction("prey_output_location", prey_output_location)
    fn.setMessageOutput("prey_location_message")

    fn = agent.newRTCFunction("prey_avoid_pred", prey_avoid_pred)
    fn.setMessageInput("predator_location_message")

    fn = agent.newRTCFunction("prey_flock", prey_flock)
    fn.setMessageInput("prey_location_message")

    fn = agent.newRTCFunction("prey_move", prey_move)

    fn = agent.newRTCFunction("prey_eaten", prey_eaten)
    fn.setMessageInput("predator_location_message")
    fn.setMessageOutput("prey_eaten_message")
    fn.setMessageOutputOptional(True)
    fn.setAllowAgentDeath(True)

    #fn = agent.newRTCFunction("prey_eat_or_starve", prey_eat_or_starve)
    #fn.setMessageInput("grass_eaten_message")
    #fn.setAllowAgentDeath(True)
    
    fn = agent.newRTCFunction("prey_reproduction", prey_reproduction)
    fn.setAgentOutput("prey", "default")
    
    """
      Predator agent
    """
    # Create the agent
    agent = model.newAgent("predator")

    # Assign its variables
    agent.newVariableFloat("x")
    agent.newVariableFloat("y")
    agent.newVariableFloat("vx")
    agent.newVariableFloat("vy")
    agent.newVariableFloat("steer_x")
    agent.newVariableFloat("steer_y")
    agent.newVariableInt("life")
    agent.newVariableFloat("type")
    
    # Assign its functions
    fn = agent.newRTCFunction("pred_output_location", pred_output_location)
    fn.setMessageOutput("predator_location_message")
    fn = agent.newRTCFunction("pred_follow_prey", pred_follow_prey)
    fn.setMessageInput("prey_location_message")
    fn = agent.newRTCFunction("pred_avoid", pred_avoid)
    fn.setMessageInput("predator_location_message")
    fn = agent.newRTCFunction("pred_move", pred_move)
    fn = agent.newRTCFunction("pred_eat_or_starve", pred_eat_or_starve)
    fn.setMessageInput("prey_eaten_message")
    fn.setAllowAgentDeath(True)
    fn = agent.newRTCFunction("pred_reproduction", pred_reproduction)
    fn.setAgentOutput("predator", "default")
    
    

# Define Agent Function Behaviours

The agent function behaviours must be defined using a small subset of the CUDA C++ programming language. A brief summary of the language is given below:

| Action | Code format |
| :--- | :--- |
| Write a comment | `// This will be a comment` |
| Declare a variable | `type variable_name = value;` |
| Declare a constant | `const type variable_name = value;` |
| Get the value of an environment property| `FLAMEGPU->environment.getProperty<type>("property_name");` |
| Iterate & access input messages | `for (const auto& msg :  FLAMEGPU->message_in) { ` <br>&nbsp;&nbsp; `// Each message is now accessible inside this loop via msg` <br> `}`
| Read an input message variable value | `msg.getVariable<type>("variable_name");` |
| Set an output message variable value | `FLAMEGPU->message_out.setVariable<type>("variable_name", value_to_set_variable_to);` |
| Generate a random float in the range [0,1) | `FLAMEGPU->random.uniform<float>()` |
| Function leaves agent alive | `return flamegpu::ALIVE` |
| Function kills agent | `return flamegpu::DEAD` |

We recognise that this is a python tutorial, so if you are having any trouble at all with the agent function syntax please do feel free to ask for help!

It is not important to read every single agent function definition below, just try to familiarise yourself with their structure a bit as we will be writing some of these later in the tutorial!

In [ ]:
"""
  outputdata agent function for Boid agents, which outputs publicly visible properties to a message list
"""
pred_output_location = r"""
FLAMEGPU_AGENT_FUNCTION(pred_output_location, flamegpu::MessageNone, flamegpu::MessageBruteForce) {
    const flamegpu::id_t id = FLAMEGPU->getID();
    const float x = FLAMEGPU->getVariable<float>("x");
    const float y = FLAMEGPU->getVariable<float>("y");
    FLAMEGPU->message_out.setVariable<int>("id", id);
    FLAMEGPU->message_out.setVariable<float>("x", x);
    FLAMEGPU->message_out.setVariable<float>("y", y);

    return flamegpu::ALIVE;
}
"""

"""
  inputdata agent function for Boid agents, which reads data from neighbouring Boid agents, to perform the boid flocking model.
"""
pred_follow_prey = r"""
FLAMEGPU_AGENT_FUNCTION(pred_follow_prey, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
    const float PRED_PREY_INTERACTION_RADIUS = FLAMEGPU->environment.getProperty<float>("PRED_PREY_INTERACTION_RADIUS");
    // Fetch the predator's position
    const float predator_x = FLAMEGPU->getVariable<float>("x");
    const float predator_y = FLAMEGPU->getVariable<float>("y");

    // Find the closest prey by iterating the prey_location messages
    float closest_prey_x = 0.0f;
    float closest_prey_y = 0.0f;
    float closest_prey_distance = PRED_PREY_INTERACTION_RADIUS;
    int is_a_prey_in_range = 0;

    for (const auto& msg : FLAMEGPU->message_in) {
        // Fetch prey location
        const float prey_x = msg.getVariable<float>("x");
        const float prey_y = msg.getVariable<float>("y");

        // Check if prey is within sight range of predator
        const float dx = predator_x - prey_x;
        const float dy = predator_y - prey_y;
        const float separation = sqrt(dx * dx + dy * dy);

        if (separation < closest_prey_distance) {
            closest_prey_x = prey_x;
            closest_prey_y = prey_y;
            closest_prey_distance = separation;
            is_a_prey_in_range = 1;
        }
    }

    // If there was a prey in range, steer the predator towards it
    if (is_a_prey_in_range) {
        const float steer_x = closest_prey_x - predator_x;
        const float steer_y = closest_prey_y - predator_y;
        FLAMEGPU->setVariable<float>("steer_x", steer_x);
        FLAMEGPU->setVariable<float>("steer_y", steer_y);
    }

    return flamegpu::ALIVE;
}
"""

pred_avoid = r"""
FLAMEGPU_AGENT_FUNCTION(pred_avoid, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
    const float SAME_SPECIES_AVOIDANCE_RADIUS = FLAMEGPU->environment.getProperty<float>("SAME_SPECIES_AVOIDANCE_RADIUS");
    // Fetch this predator's position
    const float predator_x = FLAMEGPU->getVariable<float>("x");
    const float predator_y = FLAMEGPU->getVariable<float>("y");
    float avoid_velocity_x = 0.0f;
    float avoid_velocity_y = 0.0f;

    // Add a steering factor away from each other predator. Strength increases with closeness.
    for (const auto& msg : FLAMEGPU->message_in) {
        // Fetch location of other predator
        const float other_predator_x = msg.getVariable<float>("x");
        const float other_predator_y = msg.getVariable<float>("y");

        // Check if the two predators are within interaction radius
        const float dx = predator_x - other_predator_x;
        const float dy = predator_y - other_predator_y;
        const float separation = sqrt(dx * dx + dy * dy);

        if (separation < SAME_SPECIES_AVOIDANCE_RADIUS && separation > 0.0f) {
            avoid_velocity_x += SAME_SPECIES_AVOIDANCE_RADIUS / separation * dx;
            avoid_velocity_y += SAME_SPECIES_AVOIDANCE_RADIUS / separation * dy;
        }
    }

    float steer_x = FLAMEGPU->getVariable<float>("steer_x");
    float steer_y = FLAMEGPU->getVariable<float>("steer_y");
    steer_x += avoid_velocity_x;
    steer_y += avoid_velocity_y;
    FLAMEGPU->setVariable<float>("steer_x", steer_x);
    FLAMEGPU->setVariable<float>("steer_y", steer_y);

    return flamegpu::ALIVE;
}
"""

pred_move = r"""
FLAMEGPU_AGENT_FUNCTION(pred_move, flamegpu::MessageNone, flamegpu::MessageNone) {
    const float MIN_POSITION = FLAMEGPU->environment.getProperty<float>("MIN_POSITION");
    const float MAX_POSITION = FLAMEGPU->environment.getProperty<float>("MAX_POSITION");
    const float DELTA_TIME = FLAMEGPU->environment.getProperty<float>("DELTA_TIME");
    const float PRED_SPEED_ADVANTAGE = FLAMEGPU->environment.getProperty<float>("PRED_SPEED_ADVANTAGE");
    float predator_x = FLAMEGPU->getVariable<float>("x");
    float predator_y = FLAMEGPU->getVariable<float>("y");
    float predator_vx = FLAMEGPU->getVariable<float>("vx");
    float predator_vy = FLAMEGPU->getVariable<float>("vy");
    const float predator_steer_x = FLAMEGPU->getVariable<float>("steer_x");
    const float predator_steer_y = FLAMEGPU->getVariable<float>("steer_y");
    const float predator_life = FLAMEGPU->getVariable<int>("life");

    // Integrate steering forces and cap velocity
    predator_vx += predator_steer_x;
    predator_vy += predator_steer_y;

    float speed = sqrt(predator_vx * predator_vx + predator_vy * predator_vy);
    if (speed > 1.0f) {
        predator_vx /= speed;
        predator_vy /= speed;
    }

    // Integrate velocity
    predator_x += predator_vx * DELTA_TIME * PRED_SPEED_ADVANTAGE;
    predator_y += predator_vy * DELTA_TIME * PRED_SPEED_ADVANTAGE;

    // Bound the position within the environment 
    predator_x = predator_x < MIN_POSITION ? MIN_POSITION : predator_x;
    predator_x = predator_x > MAX_POSITION ? MAX_POSITION : predator_x;
    predator_y = predator_y < MIN_POSITION ? MIN_POSITION : predator_y;
    predator_y = predator_y > MAX_POSITION ? MAX_POSITION : predator_y;

    // Update agent state
    FLAMEGPU->setVariable<float>("x", predator_x);
    FLAMEGPU->setVariable<float>("y", predator_y);
    FLAMEGPU->setVariable<float>("vx", predator_vx);
    FLAMEGPU->setVariable<float>("vy", predator_vy);

    // Reduce life by one unit of energy
    FLAMEGPU->setVariable<int>("life", predator_life - 1);

    return flamegpu::ALIVE;
}
"""

pred_eat_or_starve = r"""
FLAMEGPU_AGENT_FUNCTION(pred_eat_or_starve, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
    const flamegpu::id_t predator_id = FLAMEGPU->getID();
    int predator_life = FLAMEGPU->getVariable<int>("life");
    int isDead = 0;

    // Iterate prey_eaten messages to see if this predator ate a prey
    for (const auto& msg : FLAMEGPU->message_in) {
        if (msg.getVariable<int>("pred_id") == predator_id) {
            predator_life += FLAMEGPU->environment.getProperty<unsigned int>("GAIN_FROM_FOOD_PREDATOR");
        }
    }

    // Update agent state
    FLAMEGPU->setVariable<int>("life", predator_life);

    // Did the predator starve?
    if (predator_life < 1) {
        isDead = 1;
    }

    return isDead ? flamegpu::DEAD : flamegpu::ALIVE;
}
"""

pred_reproduction = r"""
FLAMEGPU_AGENT_FUNCTION(pred_reproduction, flamegpu::MessageNone, flamegpu::MessageNone) {
    const float BOUNDS_WIDTH = FLAMEGPU->environment.getProperty<float>("BOUNDS_WIDTH");
    float random = FLAMEGPU->random.uniform<float>();
    const int currentLife = FLAMEGPU->getVariable<int>("life");
    if (random < FLAMEGPU->environment.getProperty<float>("REPRODUCE_PRED_PROB")) {
        int id = FLAMEGPU->random.uniform<float>() * (float)INT_MAX;
        float x = FLAMEGPU->random.uniform<float>() * BOUNDS_WIDTH - BOUNDS_WIDTH / 2.0f;
        float y = FLAMEGPU->random.uniform<float>() * BOUNDS_WIDTH - BOUNDS_WIDTH / 2.0f;
        float vx = FLAMEGPU->random.uniform<float>() * 2 - 1;
        float vy = FLAMEGPU->random.uniform<float>() * 2 - 1;

        FLAMEGPU->setVariable<int>("life", currentLife / 2);

        FLAMEGPU->agent_out.setVariable<float>("x", x);
        FLAMEGPU->agent_out.setVariable<float>("y", y);
        FLAMEGPU->agent_out.setVariable<float>("type", 0.0f);
        FLAMEGPU->agent_out.setVariable<float>("vx", vx);
        FLAMEGPU->agent_out.setVariable<float>("vy", vy);
        FLAMEGPU->agent_out.setVariable<float>("steer_x", 0.0f);
        FLAMEGPU->agent_out.setVariable<float>("steer_y", 0.0f);
        FLAMEGPU->agent_out.setVariable<int>("life", currentLife / 2);

    }
    return flamegpu::ALIVE;
}
"""

prey_output_location = r"""
FLAMEGPU_AGENT_FUNCTION(prey_output_location, flamegpu::MessageNone, flamegpu::MessageBruteForce) {
    const flamegpu::id_t id = FLAMEGPU->getID();
    const float x = FLAMEGPU->getVariable<float>("x");
    const float y = FLAMEGPU->getVariable<float>("y");
    FLAMEGPU->message_out.setVariable<int>("id", id);
    FLAMEGPU->message_out.setVariable<float>("x", x);
    FLAMEGPU->message_out.setVariable<float>("y", y);
    return flamegpu::ALIVE;
}
"""

prey_avoid_pred = r"""
FLAMEGPU_AGENT_FUNCTION(prey_avoid_pred, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
    const float PRED_PREY_INTERACTION_RADIUS = FLAMEGPU->environment.getProperty<float>("PRED_PREY_INTERACTION_RADIUS");
    // Fetch this prey's position
    const float prey_x = FLAMEGPU->getVariable<float>("x");
    const float prey_y = FLAMEGPU->getVariable<float>("y");
    float avoid_velocity_x = 0.0f;
    float avoid_velocity_y = 0.0f;

    // Add a steering factor away from each predator. Strength increases with closeness.
    for (const auto& msg : FLAMEGPU->message_in) {
        // Fetch location of predator
        const float predator_x = msg.getVariable<float>("x");
        const float predator_y = msg.getVariable<float>("y");

        // Check if the two predators are within interaction radius
        const float dx = prey_x - predator_x;
        const float dy = prey_y - predator_y;
        const float distance = sqrt(dx * dx + dy * dy);

        if (distance < PRED_PREY_INTERACTION_RADIUS) {
            // Steer the prey away from the predator
            avoid_velocity_x += (PRED_PREY_INTERACTION_RADIUS / distance) * dx;
            avoid_velocity_y += (PRED_PREY_INTERACTION_RADIUS / distance) * dy;
        }
    }

    // Update agent state 
    FLAMEGPU->setVariable<float>("steer_x", avoid_velocity_x);
    FLAMEGPU->setVariable<float>("steer_y", avoid_velocity_y);

    return flamegpu::ALIVE;
}
"""

prey_flock = r"""
FLAMEGPU_AGENT_FUNCTION(prey_flock, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
    const float PREY_GROUP_COHESION_RADIUS = FLAMEGPU->environment.getProperty<float>("PREY_GROUP_COHESION_RADIUS");
    const float SAME_SPECIES_AVOIDANCE_RADIUS = FLAMEGPU->environment.getProperty<float>("SAME_SPECIES_AVOIDANCE_RADIUS");
    const flamegpu::id_t prey_id = FLAMEGPU->getID();
    const float prey_x = FLAMEGPU->getVariable<float>("x");
    const float prey_y = FLAMEGPU->getVariable<float>("y");

    float group_centre_x = 0.0f;
    float group_centre_y = 0.0f;
    float group_velocity_x = 0.0f;
    float group_velocity_y = 0.0f;
    float avoid_velocity_x = 0.0f;
    float avoid_velocity_y = 0.0f;
    int group_centre_count = 0;

    for (const auto& msg : FLAMEGPU->message_in) {
        const int   other_prey_id = msg.getVariable<int>("id");
        const float other_prey_x = msg.getVariable<float>("x");
        const float other_prey_y = msg.getVariable<float>("y");
        const float dx = prey_x - other_prey_x;
        const float dy = prey_y - other_prey_y;
        const float separation = sqrt(dx * dx + dy * dy);

        if (separation < PREY_GROUP_COHESION_RADIUS && prey_id != other_prey_id) {
            group_centre_x += other_prey_x;
            group_centre_y += other_prey_y;
            group_centre_count += 1;

            // Avoidance behaviour
            if (separation < SAME_SPECIES_AVOIDANCE_RADIUS) {
                // Was a check for separation > 0 in original - redundant?
                avoid_velocity_x += SAME_SPECIES_AVOIDANCE_RADIUS / separation * dx;
                avoid_velocity_y += SAME_SPECIES_AVOIDANCE_RADIUS / separation * dy;
            }
        }
    }

    // Compute group centre as the average of the nearby prey positions and a velocity to move towards the group centre
    if (group_centre_count > 0) {
        group_centre_x /= group_centre_count;
        group_centre_y /= group_centre_count;
        group_velocity_x = group_centre_x - prey_x;
        group_velocity_y = group_centre_y - prey_y;
    }

    float prey_steer_x = FLAMEGPU->getVariable<float>("steer_x");
    float prey_steer_y = FLAMEGPU->getVariable<float>("steer_y");
    prey_steer_x += group_velocity_x + avoid_velocity_x;
    prey_steer_y += group_velocity_y + avoid_velocity_y;
    FLAMEGPU->setVariable<float>("steer_x", prey_steer_x);
    FLAMEGPU->setVariable<float>("steer_y", prey_steer_y);

    return flamegpu::ALIVE;
}
"""

prey_move = r"""
FLAMEGPU_AGENT_FUNCTION(prey_move, flamegpu::MessageNone, flamegpu::MessageNone) {
    const float MIN_POSITION = FLAMEGPU->environment.getProperty<float>("MIN_POSITION");
    const float MAX_POSITION = FLAMEGPU->environment.getProperty<float>("MAX_POSITION");
    const float DELTA_TIME = FLAMEGPU->environment.getProperty<float>("DELTA_TIME");
    float prey_x = FLAMEGPU->getVariable<float>("x");
    float prey_y = FLAMEGPU->getVariable<float>("y");
    float prey_vx = FLAMEGPU->getVariable<float>("vx");
    float prey_vy = FLAMEGPU->getVariable<float>("vy");
    const float prey_steer_x = FLAMEGPU->getVariable<float>("steer_x");
    const float prey_steer_y = FLAMEGPU->getVariable<float>("steer_y");
    const float prey_life = FLAMEGPU->getVariable<int>("life");

    // Integrate steering forces and cap velocity
    prey_vx += prey_steer_x;
    prey_vy += prey_steer_y;

    float speed = sqrt(prey_vx * prey_vx + prey_vy * prey_vy);
    if (speed > 1.0f) {
        prey_vx /= speed;
        prey_vy /= speed;
    }

    // Integrate velocity
    prey_x += prey_vx * DELTA_TIME;
    prey_y += prey_vy * DELTA_TIME;

    // Bound the position within the environment - can this be moved
    prey_x = prey_x < MIN_POSITION ? MIN_POSITION : prey_x;
    prey_x = prey_x > MAX_POSITION ? MAX_POSITION : prey_x;
    prey_y = prey_y < MIN_POSITION ? MIN_POSITION : prey_y;
    prey_y = prey_y > MAX_POSITION ? MAX_POSITION : prey_y;


    // Update agent state
    FLAMEGPU->setVariable<float>("x", prey_x);
    FLAMEGPU->setVariable<float>("y", prey_y);
    FLAMEGPU->setVariable<float>("vx", prey_vx);
    FLAMEGPU->setVariable<float>("vy", prey_vy);

    // Reduce life by one unit of energy
    FLAMEGPU->setVariable<int>("life", prey_life - 1);

    return flamegpu::ALIVE;
}
"""

prey_eaten = r"""
FLAMEGPU_AGENT_FUNCTION(prey_eaten, flamegpu::MessageBruteForce, flamegpu::MessageBruteForce) {
    const float PRED_KILL_DISTANCE = FLAMEGPU->environment.getProperty<float>("PRED_KILL_DISTANCE");
    const flamegpu::id_t id = FLAMEGPU->getID();
    int eaten = 0;
    int predator_id = -1;
    float closest_pred = PRED_KILL_DISTANCE;
    const float prey_x = FLAMEGPU->getVariable<float>("x");
    const float prey_y = FLAMEGPU->getVariable<float>("y");

    // Iterate predator_location messages to find the closest predator
    for (const auto& msg : FLAMEGPU->message_in) {
        // Fetch location of predator
        const float predator_x = msg.getVariable<float>("x");
        const float predator_y = msg.getVariable<float>("y");

        // Check if the two predators are within interaction radius
        const float dx = prey_x - predator_x;
        const float dy = prey_y - predator_y;
        const float distance = sqrt(dx * dx + dy * dy);

        if (distance < closest_pred) {
            predator_id = msg.getVariable<int>("id");
            closest_pred = distance;
            eaten = 1;
        }
    }

    if (eaten) {
        FLAMEGPU->message_out.setVariable<int>("id", id);
        FLAMEGPU->message_out.setVariable<int>("pred_id", predator_id);
    }

    return eaten ? flamegpu::DEAD : flamegpu::ALIVE;
}
"""

prey_reproduction = r"""
FLAMEGPU_AGENT_FUNCTION(prey_reproduction, flamegpu::MessageNone, flamegpu::MessageNone) {
    const float REPRODUCE_PREY_PROB = FLAMEGPU->environment.getProperty<float>("REPRODUCE_PREY_PROB");
    const float BOUNDS_WIDTH = FLAMEGPU->environment.getProperty<float>("BOUNDS_WIDTH");
    float random = FLAMEGPU->random.uniform<float>();
    const int currentLife = FLAMEGPU->getVariable<int>("life");
    if (random < FLAMEGPU->environment.getProperty<float>("REPRODUCE_PREY_PROB")) {
        int id = FLAMEGPU->random.uniform<float>() * (float)INT_MAX;
        float x = FLAMEGPU->random.uniform<float>() * BOUNDS_WIDTH - BOUNDS_WIDTH / 2.0f;
        float y = FLAMEGPU->random.uniform<float>() * BOUNDS_WIDTH - BOUNDS_WIDTH / 2.0f;
        float vx = FLAMEGPU->random.uniform<float>() * 2 - 1;
        float vy = FLAMEGPU->random.uniform<float>() * 2 - 1;

        FLAMEGPU->setVariable<int>("life", currentLife / 2);

        FLAMEGPU->agent_out.setVariable<float>("x", x);
        FLAMEGPU->agent_out.setVariable<float>("y", y);
        FLAMEGPU->agent_out.setVariable<float>("type", 1.0f);
        FLAMEGPU->agent_out.setVariable<float>("vx", vx);
        FLAMEGPU->agent_out.setVariable<float>("vy", vy);
        FLAMEGPU->agent_out.setVariable<float>("steer_x", 0.0f);
        FLAMEGPU->agent_out.setVariable<float>("steer_y", 0.0f);
        FLAMEGPU->agent_out.setVariable<int>("life", currentLife / 2);

    }
    return flamegpu::ALIVE;
}
"""

grass_output_location = r"""
FLAMEGPU_AGENT_FUNCTION(grass_output_location, flamegpu::MessageNone, flamegpu::MessageBruteForce) {
    // Exercise 3.1 : Set the variables for the grass_location message
    const flamegpu::id_t id = FLAMEGPU->getID();
    const float x = FLAMEGPU->getVariable<float>("x");
    const float y = FLAMEGPU->getVariable<float>("y");
    FLAMEGPU->message_out.setVariable<int>("id", id);
    FLAMEGPU->message_out.setVariable<float>("x", x);
    FLAMEGPU->message_out.setVariable<float>("y", y);
    return flamegpu::ALIVE;
}


"""

grass_eaten = r"""
FLAMEGPU_AGENT_FUNCTION(grass_eaten, flamegpu::MessageBruteForce, flamegpu::MessageBruteForce) {
    const float grass_x = FLAMEGPU->getVariable<float>("x");
    const float grass_y = FLAMEGPU->getVariable<float>("y");
    int available = FLAMEGPU->getVariable<int>("available");
    if (available) { 

        int prey_id = -1;
        float closest_prey = FLAMEGPU->environment.getProperty<float>("GRASS_EAT_DISTANCE");
        int eaten = 0;

        // Iterate predator_location messages to find the closest predator
        for (const auto& msg : FLAMEGPU->message_in) {
            // Fetch location of prey
            const float prey_x = msg.getVariable<float>("x");
            const float prey_y = msg.getVariable<float>("y");

            // Check if the two preys are within interaction radius
            const float dx = grass_x - prey_x;
            const float dy = grass_y - prey_y;
            const float distance = sqrt(dx*dx + dy*dy);

            if (distance < closest_prey) {
                prey_id = msg.getVariable<int>("id");
                closest_prey= distance;
                eaten = 1;
            }
        }

        if (eaten) {
            // Add grass eaten message
            FLAMEGPU->message_out.setVariable<int>("id", FLAMEGPU->getID());
            FLAMEGPU->message_out.setVariable<int>("prey_id", prey_id);
           
            // Update grass agent variables
            FLAMEGPU->setVariable<int>("dead_cycles", 0);
            FLAMEGPU->setVariable<int>("available", 0);
            FLAMEGPU->setVariable<float>("type", 3.0f);
        }
    }
    return flamegpu::ALIVE;
}
"""

prey_eat_or_starve = r"""
FLAMEGPU_AGENT_FUNCTION(prey_eat_or_starve, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
    int isDead = 0;
    const flamegpu::id_t id = FLAMEGPU->getID();
    const int life = FLAMEGPU->getVariable<int>("life");

    // Iterate the grass eaten messages 
    for (const auto& msg : FLAMEGPU->message_in)
    {
        // If the grass eaten message indicates that this prey ate some grass then increase the preys life by adding energy
        if (id == msg.getVariable<int>("prey_id")) {
            FLAMEGPU->setVariable<int>("life", life + FLAMEGPU->environment.getProperty<unsigned int>("GAIN_FROM_FOOD_PREY"));
        }
    }

    // If the life has reduced to 0 then the prey should die or starvation 
    if (FLAMEGPU->getVariable<int>("life") < 1)
        isDead = 1;

    return isDead ? flamegpu::DEAD : flamegpu::ALIVE;
}
"""

grass_growth = r"""
FLAMEGPU_AGENT_FUNCTION(grass_growth, flamegpu::MessageNone, flamegpu::MessageNone) {
    const int dead_cycles = FLAMEGPU->getVariable<int>("dead_cycles");
    int new_dead_cycles = dead_cycles + 1;
    if (dead_cycles == FLAMEGPU->environment.getProperty<unsigned int>("GRASS_REGROW_CYCLES")) {
        FLAMEGPU->setVariable<int>("dead_cycles", 0);
        FLAMEGPU->setVariable<int>("available", 1);
        FLAMEGPU->setVariable<float>("type", 2.0f);
    } 

    const int available = FLAMEGPU->getVariable<int>("available");
    if (available == 0) {
        FLAMEGPU->setVariable<int>("dead_cycles", new_dead_cycles);
    } 

    return flamegpu::ALIVE;
}
"""

class population_tracker(pyflamegpu.HostFunction):
    def __init__(self, has_grass):
        super().__init__();  # Mandatory if we are defining __init__ ourselves
        # Local, so value is maintained between calls to calculate_convergence::run
        self.pred_count = []
        self.prey_count = []
        self.grass_count = []
        self.has_grass = has_grass

    def run(self, FLAMEGPU):
        # Reduce force and overlap
        self.pred_count.append(FLAMEGPU.agent("predator").count())
        self.prey_count.append(FLAMEGPU.agent("prey").count())
        if (self.has_grass):
            self.grass_count.append(FLAMEGPU.agent("grass").countInt("available", 1))
        else:
            self.grass_count.append(0)

    def plot(self):
        plt.figure(figsize=(16,10))
        plt.rcParams.update({'font.size': 18})
        plt.xlabel("Step")
        plt.ylabel("Population")
        plt.plot(range(0, len(self.pred_count)), self.pred_count, 'r', label="Predators")
        plt.plot(range(0, len(self.prey_count)), self.prey_count, 'b', label="Prey")
        plt.plot(range(0, len(self.grass_count)), self.grass_count, 'g', label="Grass")
        plt.legend()
        plt.show()

    def reset(self):
        self.pred_count = []
        self.prey_count = []
        self.grass_count = []

# Keeping Track of Model Outputs

It is possible for FLAME GPU 2 to log the variables of each agent at every time step. This can be useful for post processing of simulation data but is usually more information than is required. For the predator prey model we are interested only in the population counts and as such we can use FLAME GPUs init, step and exit functions to run host code for logging or tracking information about the state of the model. Init functions are executed once at the start of a simulation, [step functions](https://docs.flamegpu.com/guide/3-behaviour-definition/6-host-functions.html) are run after each simulation step and exit functions are run at the end of the simulation. In this case, we have used FLAME GPU 2's `HostFunction` to track the populations at each step and plot them. This is shown at the end of the previous code segment.

# Define Execution Order

The final piece of the puzzle is function layers which define the order functions should be executed in. Functions in the same layer may be executed concurrently. It is also possible to automatically generate the execution layers to optimise performance using concurrent execution. This is achieved using specification of the dependencies between the functions. We will not cover this in this tutorial, but if you are interested, the documentation is available [here](https://docs.flamegpu.com/guide/5-running-a-sim/1-defining-execution-order.html). The function layers from this model are shown below:

In [ ]:
def define_execution_order(model):
    """
      Control flow
    """    
    layer = model.newLayer()
    layer.addAgentFunction("prey", "prey_output_location")
    layer.addAgentFunction("predator", "pred_output_location")
    
    layer = model.newLayer()
    layer.addAgentFunction("predator", "pred_follow_prey")
    layer.addAgentFunction("prey", "prey_avoid_pred")
    
    layer = model.newLayer()
    layer.addAgentFunction("prey", "prey_flock")
    layer.addAgentFunction("predator", "pred_avoid")
    
    layer = model.newLayer()
    layer.addAgentFunction("prey", "prey_move")
    layer.addAgentFunction("predator", "pred_move")
    
    layer = model.newLayer()
    layer.addAgentFunction("prey", "prey_eaten")
    
    layer = model.newLayer()
    #layer.addAgentFunction("prey", "prey_eat_or_starve")
    layer.addAgentFunction("predator", "pred_eat_or_starve")
    
    layer = model.newLayer()
    layer.addAgentFunction("predator", "pred_reproduction")
    layer.addAgentFunction("prey", "prey_reproduction")

# Create Populations

The initial agent data for the model can either be specified within the main python file or in an XML file. In this tutorial, we demonstrate the agents being generated programmatically. The program generates the specified number of predator and prey agents in random positions (``x``,``y``) between [-1,1] with random velocities (``vx``,``vy``) between [-1,1]. Each predator is given an amount of energy (life) which is randomly selected from the interval of [0,40]. Prey are initialised with random positions between [-1, 1], a velocity of 0 and energy randomly selected in the interval [0, 50]. 

In [ ]:
def initialise_simulation(num_prey, num_predators, num_grass, seed):
    model = create_model()
    define_messages(model)
    define_agents(model)
    define_environment(model)
    define_execution_order(model)

    # Set up a population tracker for logging/plotting
    pop_tracker = population_tracker(num_grass > 0)
    model.addStepFunction(pop_tracker)
    
    """
      Create Model Runner
    """   
    cudaSimulation = pyflamegpu.CUDASimulation(model)
    
    # Apply a simulation seed
    if seed is not None:
        cudaSimulation.SimulationConfig().random_seed = seed
        cudaSimulation.applyConfig()
    
    """
      Initialise Model
    """
    # If no xml model file was is provided, generate a population programmatically.
    if not cudaSimulation.SimulationConfig().input_file:
        # Uniformly distribute agents within space, with uniformly distributed initial velocity.
        # Using random seed
        random.seed(cudaSimulation.SimulationConfig().random_seed)
    
        # Initialise prey agents
        preyPopulation = pyflamegpu.AgentVector(model.Agent("prey"), num_prey)
        for i in range(0, num_prey):
            prey = preyPopulation[i]
            prey.setVariableFloat("x", random.uniform(-1.0, 1.0))
            prey.setVariableFloat("y", random.uniform(-1.0, 1.0))
            prey.setVariableFloat("vx", random.uniform(-1.0, 1.0))
            prey.setVariableFloat("vy", random.uniform(-1.0, 1.0))
            prey.setVariableFloat("steer_x", 0.0)
            prey.setVariableFloat("steer_y", 0.0)
            prey.setVariableFloat("type", 1.0)
            prey.setVariableInt("life", random.randint(0, 50))
      
        # Initialise predator agents
        predatorPopulation = pyflamegpu.AgentVector(model.Agent("predator"), num_predators)
        for i in range(0, num_predators):
            predator = predatorPopulation[i]
            predator.setVariableFloat("x", random.uniform(-1.0, 1.0))
            predator.setVariableFloat("y", random.uniform(-1.0, 1.0))
            predator.setVariableFloat("vx", random.uniform(-1.0, 1.0))
            predator.setVariableFloat("vy", random.uniform(-1.0, 1.0))
            predator.setVariableFloat("steer_x", 0.0)
            predator.setVariableFloat("steer_y", 0.0)
            predator.setVariableFloat("type", 0.0)
            predator.setVariableInt("life", random.randint(0, 5))

    cudaSimulation.setPopulationData(predatorPopulation)
    cudaSimulation.setPopulationData(preyPopulation)
        
    return [cudaSimulation, pop_tracker]

# Set the number of steps and run!

Depending on the parameters this could take anywhere from a few seconds to a few minutes.

In [ ]:
def run_simulation():
    """
      Execution
    """
    # Initialise the simulation
    [cudaSimulation, pop_tracker] = initialise_simulation(num_prey = 200, num_predators = 50, num_grass = 0, seed = 64)
    cudaSimulation.SimulationConfig().steps = 1600

    # Run the simulation
    pop_tracker.reset()
    cudaSimulation.simulate()
    pop_tracker.plot()

run_simulation()

# Exercise 01: Running FLAME GPU 2

To run the simulation, you can either run each code cell sequentially from the top of the document, or you can click **Runtime** then **Run All** in the top menu. If you update a code segment, you will need to run that segment again to save the changes to the model, or use the **Run All** option again. The first time you run the simulation it might take a little while as the library is installed and the agent functions are compiled.

Try using **Run All** now to run the simulation. You should see a graph appear showing the population changes over time.

# Exercise 02: Changing the Model's Behaviour

Let's change the parameters of the simulation and see how this affects the results. To start, try changing the `GAIN_FROM_FOOD_PREDATOR` environment variable to ``50``.

(**Hint:** You can collapse the individual sections of code so you don't have to scroll so far up and down)

Now try modifying the initial populations which are passed to the `initialise_simulation` function in the section "Set the number of steps and run!"

See if you can produce oscillating predator and prey populations.




# Exercise 03: Extending the Model

In this exercise, we are going to extend our model to include grass to familiarise you with how agents and their behaviours are implemented within FLAMEGPU2. The grass will provide a source of food for the prey which should behave according to the following rules:

- A prey’s energy is reduced by 1 unit each time it moves.
- A prey’s energy increases each time it eats grass.
- A prey dies if it has run out of energy.
- Once eaten, grass regrows after a fixed number of iterations.

With respect to the implementation this will require , a fourth and fifth message type as grass should only be eaten by prey within a certain certain radius. The mechanism for "grazing" is as follows:

1. Grass agents which currently are currently available read the positions of the prey from the location messages to locate the nearest prey.
2. If a prey was in eating range, a grass_eaten_message message is output. Note that if there is more than 1 prey on within the minimum distance, the grass agent should be eaten by the closest prey.
3. Grass agents then modify the active agent variable to indicate they are in a regrowth period and cannot be eaten at this time.
4. Prey agents read the grass_eaten_message messages and check to see if any match their ID. They then increase their energy accordingly.
5. Prey agents die if they do not have enough life/energy.

In the following exercises you will add the grass agent to the model:

<br />
<br />

## Exercise 3.1
Exercise 3.1 will add the grass agent definition.

&nbsp;&nbsp;&nbsp;&nbsp;**a)** Add a new agent type called ``grass``

&nbsp;&nbsp;&nbsp;&nbsp;**b)** Add the following variables to the ``grass`` agent:


| Type | Name |
| :--- | :--- |
| float | x |
| float | y |
| int | dead_cycles |
| int | available |
| float | type |
 
<br />
<br />


## Exercise 3.2
Exercise 3.2 will create the message types used by the grass agent. 

&nbsp;&nbsp;&nbsp;&nbsp;**a)** Add a message type called ``grass_location_message`` with the following properties:

| Type | Name |
| :--- | :--- |
| ID | id |
| float | x |
| float | y |

<br />
<br />

## Exercise 3.3
Exercise 3.3 will add functions to the ``grass`` agent. Add the following functions to the ``grass`` agent with the properties given in the table below:

| Name | Message Input | Message Output | Message Output Optional | Allow Agent Death |
| :--- | :--- | :--- | :--- | :--- | 
| grass_output_location | n/a | grass_location_message | True | n/a |
| grass_eaten | prey_location_message | grass_eaten_message | True | True |
| grass_growth | n/a | n/a | n/a | n/a |

<br />
<br />

## Exercise 3.4
Exercise 3.4 will add a grass population to the simulation. Using the ``predatorPopulation`` and ``preyPopulation`` as examples, create a ``grassPopulation`` with variables initialised with the following properties:

| Variable | Initial Value |
| :--- | :--- |
| ``x`` | ``random.uniform(-1.0, 1.0)`` |
| ``y`` | ``random.uniform(-1.0, 1.0)`` |
| ``dead_cycles`` | ``0`` |
| ``available`` | ``1`` |
| ``type`` | ``2.0`` |

<br />
<br />

## Exercise 3.5
Exercise 3.5 will include the newly created grass agent functions in the execution layers. 

&nbsp;&nbsp;&nbsp;&nbsp;**a)** Add the ``grass_output_location`` function to the layer with ``prey_output_location`` and ``predator_output_location``.

&nbsp;&nbsp;&nbsp;&nbsp;**b)** Add the ``grass_eaten`` function to the layer with ``prey_eaten``.

&nbsp;&nbsp;&nbsp;&nbsp;**c)** Add the ``grass_growth`` function to the layer with ``predator_reproduction`` and ``prey_reproduction``.

&nbsp;&nbsp;&nbsp;&nbsp;**d)** Uncomment the line ``#layer.addAgentFunction("prey", "prey_eat_or_starve")`` in ``define_execution_order`` and the lines 
  ```
  #fn = agent.newRTCFunction("prey_eat_or_starve", prey_eat_or_starve)
  #fn.setMessageInput("grass_eaten_message")
  #fn.setAllowAgentDeath(True)
  ``` 
  in ``define_agents``.

<br />
<br />

## Exercise 3.6
Exercise 3.6 will add the behaviour definition for the ``grass_output_location`` function. The function should get the ``id``, ``x`` and ``y`` variables of the agent and use them to set the ``id``, ``x`` and ``y`` variables of the output message. The other grass agent function behaviour definitions have been provided for you, so if you change the initial number of grass and run the model, you should now see the grass population changing and affecting the dynamics of the simulation!






# Exercise 3.1 Solution


```python
    """
      Exercise 3.1 - Grass agent
    """
    # Create the agent
    agent = model.newAgent("grass")

    # Assign its variables
    agent.newVariableFloat("x")
    agent.newVariableFloat("y")
    agent.newVariableInt("dead_cycles")
    agent.newVariableInt("available")
    agent.newVariableFloat("type")
```

# Exercise 3.2 Solution

```python  
    # Assign its functions
    fn = agent.newRTCFunction("grass_output_location", grass_output_location)
    fn.setMessageOutput("grass_location_message")
    fn.setMessageOutputOptional(True)

    fn = agent.newRTCFunction("grass_eaten", grass_eaten)
    fn.setMessageInput("prey_location_message")
    fn.setMessageOutput("grass_eaten_message")
    fn.setMessageOutputOptional(True)
    fn.setAllowAgentDeath(True)
    
    fn = agent.newRTCFunction("grass_growth", grass_growth)
```

# Exercise 3.3 Solution

```python  
    message = model.newMessageBruteForce("grass_location_message")
    message.newVariableID("id")
    message.newVariableFloat("x")
    message.newVariableFloat("y")
```

# Exercise 3.4 Solution

```python
        # Initialise grass agents
        grassPopulation = pyflamegpu.AgentVector(model.Agent("grass"), num_grass)
        for i in range(0, num_grass):
            grass = grassPopulation[i]
            grass.setVariableFloat("x", random.uniform(-1.0, 1.0))
            grass.setVariableFloat("y", random.uniform(-1.0, 1.0))
            grass.setVariableInt("dead_cycles", 0)
            grass.setVariableInt("available", 1)
            grass.setVariableFloat("type", 2.0)
  
    cudaSimulation.setPopulationData(predatorPopulation)
    cudaSimulation.setPopulationData(preyPopulation)
    cudaSimulation.setPopulationData(grassPopulation)
```

# Exercise 3.5 Solution

```python
def define_execution_order(model):
    """
      Control flow
    """    
    layer = model.newLayer()
    layer.addAgentFunction("prey", "prey_output_location")
    layer.addAgentFunction("predator", "pred_output_location")
    layer.addAgentFunction("grass", "grass_output_location")
    
    layer = model.newLayer()
    layer.addAgentFunction("predator", "pred_follow_prey")
    layer.addAgentFunction("prey", "prey_avoid_pred")
    
    layer = model.newLayer()
    layer.addAgentFunction("prey", "prey_flock")
    layer.addAgentFunction("predator", "pred_avoid")
    
    layer = model.newLayer()
    layer.addAgentFunction("prey", "prey_move")
    layer.addAgentFunction("predator", "pred_move")
    
    layer = model.newLayer()
    layer.addAgentFunction("grass", "grass_eaten")
    layer.addAgentFunction("prey", "prey_eaten")
    
    layer = model.newLayer()
    layer.addAgentFunction("prey", "prey_eat_or_starve")
    layer.addAgentFunction("predator", "pred_eat_or_starve")
    
    layer = model.newLayer()
    layer.addAgentFunction("predator", "pred_reproduction")
    layer.addAgentFunction("prey", "prey_reproduction")
    layer.addAgentFunction("grass", "grass_growth")
```

# Exercise 3.6 Solution

```c++
FLAMEGPU_AGENT_FUNCTION(grass_output_location, flamegpu::MessageNone, flamegpu::MessageBruteForce) {
    // Exercise 3.1 : Set the variables for the grass_location message
    const flamegpu::id_t id = FLAMEGPU->getID();
    const float x = FLAMEGPU->getVariable<float>("x");
    const float y = FLAMEGPU->getVariable<float>("y");
    FLAMEGPU->message_out.setVariable<int>("id", id);
    FLAMEGPU->message_out.setVariable<float>("x", x);
    FLAMEGPU->message_out.setVariable<float>("y", y);
    return flamegpu::ALIVE;
}
```

# Thanks for Attending!

If you have enjoyed today's tutorial and wish to continue experimenting with FLAME GPU 2, the following resources may be useful to you:

- [Software Website](https://flamegpu.com/)
- [FLAME GPU 2 on Github](https://github.com/FLAMEGPU/FLAMEGPU2/)
- [Documentation](https://docs.flamegpu.com/guide/index.html#)
- [C++ Tutorial](https://github.com/FLAMEGPU/FLAMEGPU2-tutorial-cudacpp)

If you think FLAME GPU 2 could be a good fit for your research project, please also feel free to get in touch with us via [rse@sheffield.ac.uk](rse@sheffield.ac.uk). The RSE team is also capable of providing expertise in reproducible research software, high-performance computing and training. For more information, please visit [rse.shef.ac.uk](https://rse.shef.ac.uk/)